In [2]:
import pandas as pd
dftest=pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv')
dftrain=pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv')
print(f'shape',dftest.shape)
print(f'shape2',dftrain.shape)

shape (175341, 45)
shape2 (82332, 45)


In [3]:
dftrain.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [4]:
dftrain['attack_cat'].value_counts()

attack_cat
Normal            37000
Generic           18871
Exploits          11132
Fuzzers            6062
DoS                4089
Reconnaissance     3496
Analysis            677
Backdoor            583
Shellcode           378
Worms                44
Name: count, dtype: int64

In [6]:
dftrain=dftrain.drop(['id'],axis=1)
dftest=dftest.drop(['id'],axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dftrain['attack_cat'] = le.fit_transform(dftrain['attack_cat'])
dftest['attack_cat'] = le.fit_transform(dftest['attack_cat'])

dftrain.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,6,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,6,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,6,0
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,6,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,6,0


In [8]:
cols_to_drop = ['proto', 'service', 'state']

dftrain = dftrain.drop(columns=cols_to_drop)
dftest  = dftest.drop(columns=cols_to_drop)


In [9]:
import numpy as np

# Replace infinities with NaN first
dftrain.replace([np.inf, -np.inf], np.nan, inplace=True)

# Count again after replacement
nan_counts_after_inf = dftrain.isna().sum()
print(nan_counts_after_inf[nan_counts_after_inf > 0])


Series([], dtype: int64)


In [11]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report

feature_cols = [c for c in dftrain.columns if c not in ['attack_cat','label']]


X_full = dftrain[feature_cols]
y_full = dftrain['attack_cat'] 

# Initialize the classifier
brf = BalancedRandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)
brf.fit(X_full, y_full)


BalancedRandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [12]:
# Features and labels for test set
X_test = dftest[feature_cols]
y_test = dftest['attack_cat']

# Predict on test set
y_pred = brf.predict(X_test)

# Classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

comparison_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred
})

# Show first 10 rows
print(comparison_df.head(5))
multi_classdf=comparison_df.to_csv("prediction_vs_actual.csv", index=False)


              precision    recall  f1-score   support

           0       0.05      0.08      0.06      2000
           1       0.07      0.11      0.08      1746
           2       0.33      0.57      0.41     12264
           3       0.85      0.41      0.56     33393
           4       0.59      0.59      0.59     18184
           5       0.97      0.98      0.98     40000
           6       0.91      0.86      0.88     56000
           7       0.75      0.79      0.77     10491
           8       0.18      0.82      0.29      1133
           9       0.03      0.88      0.05       130

    accuracy                           0.73    175341
   macro avg       0.47      0.61      0.47    175341
weighted avg       0.80      0.73      0.75    175341

   Actual  Predicted
0       6          8
1       6          3
2       6          3
3       6          3
4       6          4


In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd

# Feature columns (drop both targets)
feature_cols = [c for c in dftrain.columns if c not in ['attack_cat', 'label']]

# Training data
X_train = dftrain[feature_cols]
y_train = dftrain['label']   # binary target

# Test data
X_test = dftest[feature_cols]
y_test = dftest['label']

# Model
brf_label = BalancedRandomForestClassifier(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

# Train
brf_label.fit(X_train, y_train)

# Predict
y_pred = brf_label.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))

# Prediction vs Actual
binary_df = pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred
})

print(binary_df.head(10))


              precision    recall  f1-score   support

           0       0.77      0.98      0.86     56000
           1       0.99      0.86      0.92    119341

    accuracy                           0.90    175341
   macro avg       0.88      0.92      0.89    175341
weighted avg       0.92      0.90      0.90    175341

   Actual  Predicted
0       0          1
1       0          1
2       0          1
3       0          1
4       0          0
5       0          0
6       0          0
7       0          0
8       0          0
9       0          0


In [14]:
print(X_train.columns.tolist())


['dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']


In [15]:
# ---------------- Multi-class predictions ----------------
X_test_multi = dftest[feature_cols]
y_test_multi = dftest['attack_cat']  # multi-class target
y_pred_multi = brf.predict(X_test_multi)

multi_classdf = pd.DataFrame({
    "Actual_Label": y_test_multi,
    "Predicted_Label": y_pred_multi
})

# ---------------- Binary predictions ----------------
X_test_bin = dftest[feature_cols]
y_test_bin = dftest['label']  # binary target
y_pred_bin = brf_label.predict(X_test_bin)

binary_df = pd.DataFrame({
    "Actual_Label": y_test_bin,
    "Predicted_Label": y_pred_bin
})


In [16]:
# Multi-class confidence
multi_class_conf = brf.predict_proba(X_test)  # shape: (n_samples, n_classes)
multi_classdf['Confidence'] = multi_class_conf.max(axis=1)  # max probability per row

# Binary confidence (probability of being attack = 1)
binary_conf = brf_label.predict_proba(X_test)[:,1]
binary_df['Confidence'] = binary_conf

def severity_label(conf):
    if conf >= 0.85:
        return "High"
    elif conf >= 0.6:
        return "Medium"
    else:
        return "Low"

multi_classdf['Severity'] = multi_classdf['Confidence'].apply(severity_label)
binary_df['Severity'] = binary_df['Confidence'].apply(severity_label)


In [17]:
multi_classdf.to_csv("multi_classdf_with_confidence.csv", index=False)
binary_df.to_csv("binary_df_with_confidence.csv", index=False)


In [18]:
# --- Binary threshold ---
binary_threshold = 0.6
binary_df['Binary_Predicted'] = (binary_df['Confidence'] >= binary_threshold).astype(int)

# --- RAG ---
def assign_rag(conf):
    if conf >= 0.85:
        return "Red"
    elif conf >= 0.6:
        return "Amber"
    else:
        return "Green"

binary_df['RAG'] = binary_df['Confidence'].apply(assign_rag)

# --- Severity ---
def severity_label(conf):
    if conf >= 0.85:
        return "High"
    elif conf >= 0.6:
        return "Medium"
    else:
        return "Low"

binary_df['Severity'] = binary_df['Confidence'].apply(severity_label)

# --- Assign Multi-class Attack_Type only for predicted attacks ---
binary_df['Attack_Type'] = 0  # default 0 for no attack
mask_attack = binary_df['Binary_Predicted'] == 1
binary_df.loc[mask_attack, 'Attack_Type'] = multi_classdf.loc[mask_attack, 'Predicted_Label']

# MultiClass Confidence: only for predicted attacks, else 0
binary_df['MultiClass_Confidence'] = 0.0
binary_df.loc[mask_attack, 'MultiClass_Confidence'] = multi_classdf.loc[mask_attack, 'Confidence']

# --- Final SOC-ready DataFrame ---
soc_df = binary_df[['Actual_Label', 'Binary_Predicted', 'Attack_Type', 
                    'Confidence', 'MultiClass_Confidence', 'RAG', 'Severity']]
soc_df.head(20)

,Actual_Label,Binary_Predicted,Attack_Type,Confidence,MultiClass_Confidence,RAG,Severity
0,0,1,8,0.9200,0.320,Red,High
1,0,1,3,0.9600,0.310,Red,High
2,0,1,3,0.9900,0.480,Red,High
3,0,1,3,0.8100,0.355,Amber,Medium
4,0,0,0,0.2900,0.000,Green,Low
5,0,0,0,0.3300,0.000,Green,Low
6,0,0,0,0.1950,0.000,Green,Low
7,0,0,0,0.4750,0.000,Green,Low
8,0,0,0,0.4300,0.000,Green,Low
9,0,0,0,0.4450,0.000,Green,Low


??????????????????????????????????????????????????????????????

In [19]:
# For binary model
binary_conf = brf_label.predict_proba(X_test)[:, 1]  # probability of 'attack'

# Add confidence to the dataframe
binary_df['Confidence'] = binary_conf

# For multi-class model
multi_class_conf = brf.predict_proba(X_test).max(axis=1)  # max probability across all classes
multi_classdf['Confidence'] = multi_class_conf


In [20]:

# --- Binary Model ---
# Compute probability of attack (label=1)
binary_df['Confidence'] = brf_label.predict_proba(X_test)[:, 1]

# Correct RAG function (high confidence → Red)
def assign_rag(conf):
    if conf >= 0.85:
        return 'Red'
    elif conf >= 0.6:
        return 'Amber'
    else:
        return 'Green'

# Apply RAG
binary_df['RAG'] = binary_df['Confidence'].apply(assign_rag)

# Optional: add Severity
def severity_label(conf):
    if conf >= 0.85:
        return 'High'
    elif conf >= 0.6:
        return 'Medium'
    else:
        return 'Low'

binary_df['Severity'] = binary_df['Confidence'].apply(severity_label)

# Save updated CSV
binary_df.to_csv("binary_SOC_alert_report.csv", index=False)


# --- Multi-class Model ---
# Compute max probability per row
multi_classdf['Confidence'] = brf.predict_proba(X_test).max(axis=1)

# Apply same RAG and Severity
multi_classdf['RAG'] = multi_classdf['Confidence'].apply(assign_rag)
multi_classdf['Severity'] = multi_classdf['Confidence'].apply(severity_label)


In [21]:
# --- Function to assign Severity based on RAG ---
def assign_severity(rag, attack_type=None):
    """
    Assigns severity level based on RAG.
    - Red -> High
    - Amber -> Medium
    - Green -> Low
    Optionally, you can factor in attack_type for multi-class alerts.
    """
    if rag == 'Red':
        return 'High'
    elif rag == 'Amber':
        return 'Medium'
    else:
        return 'Low'

# --- Binary alerts ---
binary_df['Severity'] = binary_df['RAG'].apply(assign_severity)

# --- Multi-class alerts ---
multi_classdf['Severity'] = multi_classdf['RAG'].apply(assign_severity)


In [22]:
# Function to convert alerts DataFrame to text
def alerts_to_text(df, max_rows=20):
    """
    Converts top alerts in a DataFrame to a readable text format.
    df: alerts DataFrame (binary or multi-class)
    max_rows: maximum number of rows to include
    """
    rows = df.head(max_rows).to_dict(orient="records")
    text = ""
    for r in rows:
        text += f"""
Attack_Type: {r.get('Predicted_Label', r.get('Attack_Type', 'N/A'))}
Confidence: {round(r.get('Confidence', 0), 2)}
RAG: {r.get('RAG', 'N/A')}
Severity: {r.get('Severity', 'N/A')}
---
"""
    return text


# ================================
# Filter high severity alerts
# ================================

high_binary = binary_df[binary_df['Severity'] == 'High']
high_multi = multi_classdf[multi_classdf['Severity'] == 'High']


# ================================
# Convert alerts to text
# ================================

binary_text = alerts_to_text(high_binary)
multi_text = alerts_to_text(high_multi)


# ================================
# Compose SOC prompt
# ================================

soc_prompt = f"""
You are a SOC Level-2 cybersecurity analyst.

Analyze the following intrusion detection alerts.

Tasks:
1. Summarize the security situation
2. Identify the most dangerous threats
3. Suggest response actions for the SOC team
4. Highlight patterns or repeated attacks

=== BINARY ALERTS ===
{binary_text}

=== MULTICLASS ALERTS ===
{multi_text}
"""


In [24]:
def build_soc_prompt(row):
    return f"""
You are a SOC analyst assistant.

Alert details:
- Prediction: {row.get('Predicted_Label', 'N/A')}
- Confidence Score: {row.get('Confidence', 0):.2f}
- Severity: {row.get('Severity', 'N/A')}
- RAG Status: {row.get('RAG', 'N/A')}

Explain:
1. What this alert means
2. Why it is risky
3. What action the SOC analyst should take

Keep the explanation concise and professional.
"""


In [25]:
# Binary model predictions
binary_conf = brf_label.predict_proba(X_test)[:, 1]
binary_pred = (binary_conf >= 0.45).astype(int)  # 1 = attack, 0 = normal

# Multi-class model predictions
multi_class_pred = brf.predict(X_test)
multi_class_conf = brf.predict_proba(X_test).max(axis=1)

# Binary DataFrame
binary_df = pd.DataFrame({
    'Actual_Label': y_test,
    'Binary_Predicted': binary_pred,
    'Confidence': binary_conf
})

# Multi-class DataFrame
multi_classdf = pd.DataFrame({
    'Predicted_Label': multi_class_pred,
    'Confidence': multi_class_conf
})

# Assign RAG
def assign_rag(conf):
    if conf >= 0.85:
        return 'Red'
    elif conf >= 0.6:
        return 'Amber'
    else:
        return 'Green'

binary_df['RAG'] = binary_df['Confidence'].apply(assign_rag)
multi_classdf['RAG'] = multi_classdf['Confidence'].apply(assign_rag)

# Assign Severity
def assign_severity(rag):
    if rag == 'Red':
        return 'High'
    elif rag == 'Amber':
        return 'Medium'
    else:
        return 'Low'

binary_df['Severity'] = binary_df['RAG'].apply(assign_severity)
multi_classdf['Severity'] = multi_classdf['RAG'].apply(assign_severity)

# Build final SOC DataFrame
final_soc_df = binary_df.copy()

# ONLY assign multi-class values if binary predicts attack (1)
final_soc_df['Attack_Type'] = [
    multi_classdf.loc[i, 'Predicted_Label'] if binary_pred[i] == 1 else 0
    for i in range(len(binary_df))
]

final_soc_df['MultiClass_Confidence'] = [
    multi_classdf.loc[i, 'Confidence'] if binary_pred[i] == 1 else 0
    for i in range(len(binary_df))
]

# SOC Playbook
def assign_playbook(severity):
    if severity == 'High':
        return "Immediate response required: Isolate affected host, block suspicious IPs, escalate to Tier-2 SOC analyst."
    elif severity == 'Medium':
        return "Investigate alert: review logs, monitor traffic, no immediate containment."
    else:
        return "Log event and continue monitoring."

final_soc_df['SOC_Playbook'] = final_soc_df['Severity'].apply(assign_playbook)

# Columns for SOC
final_soc_df = final_soc_df[
    ['Actual_Label', 'Binary_Predicted', 'Attack_Type', 
     'Confidence', 'MultiClass_Confidence', 'RAG', 'Severity', 'SOC_Playbook']
]

final_soc_df.to_csv("SOC_final_alerts_corrected.csv", index=False)
final_soc_df.head(20)

,Actual_Label,Binary_Predicted,Attack_Type,Confidence,MultiClass_Confidence,RAG,Severity,SOC_Playbook
0,0,1,8,0.9200,0.320,Red,High,Immediate response required: Isolate affected ...
1,0,1,3,0.9600,0.310,Red,High,Immediate response required: Isolate affected ...
2,0,1,3,0.9900,0.480,Red,High,Immediate response required: Isolate affected ...
3,0,1,3,0.8100,0.355,Amber,Medium,"Investigate alert: review logs, monitor traffi..."
4,0,0,0,0.2900,0.000,Green,Low,Log event and continue monitoring.
5,0,0,0,0.3300,0.000,Green,Low,Log event and continue monitoring.
6,0,0,0,0.1950,0.000,Green,Low,Log event and continue monitoring.
7,0,1,4,0.4750,0.470,Green,Low,Log event and continue monitoring.
8,0,0,0,0.4300,0.000,Green,Low,Log event and continue monitoring.
9,0,0,0,0.4450,0.000,Green,Low,Log event and continue monitoring.


In [26]:
from sklearn.metrics import confusion_matrix, classification_report

# Load your final SOC CSV
df = pd.read_csv("SOC_final_alerts_corrected.csv")

# Extract actual and predicted labels
y_true = df['Actual_Label']
y_pred = df['Binary_Predicted']

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# Classification Report
report = classification_report(y_true, y_pred)
print("\nClassification Report:\n", report)

# Optional accuracy check
accuracy = (y_true == y_pred).mean()
print(f"\nAccuracy: {accuracy:.4f}")


Confusion Matrix:
 [[ 54537   1463]
 [ 15758 103583]]

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.97      0.86     56000
           1       0.99      0.87      0.92    119341

    accuracy                           0.90    175341
   macro avg       0.88      0.92      0.89    175341
weighted avg       0.92      0.90      0.90    175341


Accuracy: 0.9018


_________________________________________________________________________________________________________________

________________ ___________________

In [36]:
import gradio as gr
import numpy as np
def predict_soc(*inputs):
    # Build input dataframe
    input_df = pd.DataFrame([inputs], columns=feature_cols)

    # ========== Binary Model ==========
    binary_probs = brf_label.predict_proba(input_df)[0]
    binary_conf = float(binary_probs[1])
    is_attack = binary_conf >= 0.25

    # ========== NORMAL TRAFFIC ==========
    if not is_attack:
        return {
            "Traffic_Type": "Normal",
            "Binary_Confidence": round(binary_conf, 3),
            "Attack_Type": "Normal",
            "MultiClass_Confidence": 0.0,
            "RAG": "Green",
            "Severity": "Low",
            "SOC_Playbook": "Log event and continue monitoring."
        }

    # ========== MULTI-CLASS MODEL ==========
    multi_probs = brf.predict_proba(input_df)[0]
    max_idx = int(np.argmax(multi_probs))
    attack_type = brf.classes_[max_idx]
    multi_conf = float(multi_probs[max_idx])

    # ========== RAG & SEVERITY ==========
    if binary_conf >= 0.6:
        rag = "Red"
        severity = "High"
        playbook = (
            "Immediate response required: isolate affected host, "
            "block malicious IPs, escalate to Tier-2 SOC analyst."
        )
    elif binary_conf >= 0.25:
        rag = "Amber"
        severity = "Medium"
        playbook = (
            "Investigate alert: review logs, monitor traffic, "
            "no immediate containment required."
        )
    else:
        rag = "Green"
        severity = "Low"
        playbook = "Log event and continue monitoring."

    return {
        "Traffic_Type": "Attack",
        "Binary_Confidence": round(binary_conf, 3),
        "Attack_Type": attack_type,
        "MultiClass_Confidence": round(multi_conf, 3),
        "RAG": rag,
        "Severity": severity,
        "SOC_Playbook": playbook
    }


In [37]:
inputs = [
    gr.Number(label=col, value=0.0)
    for col in feature_cols
]

outputs = gr.JSON(label="SOC Alert Output")

app = gr.Interface(
    fn=predict_soc,
    inputs=inputs,
    outputs=outputs,
    title="SOC Intrusion Detection System (ML-Based)",
    description=(
        "Enter numeric network flow features. "
        "The system performs real-time intrusion detection "
        "using binary + multi-class ML models."
    )
)

app.launch()


* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://12378562b53a941778.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
